In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from tqdm import tqdm
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import label_binarize
from sklearn.model_selection import train_test_split

2022-10-03 15:26:09.638692: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
labels_df = pd.read_csv('/Users/nikhil/Downloads/archive/labels.csv')

In [3]:
breeds = labels_df['breed'].value_counts()
breeds.head()

scottish_deerhound      126
maltese_dog             117
afghan_hound            116
entlebucher             115
bernese_mountain_dog    114
Name: breed, dtype: int64

In [4]:
breed_names = ['scottish_deerhound','maltese_dog','bernese_mountain_dog','entlebucher','afghan_hound']
# 
labels = labels_df[(labels_df['breed'].isin(breed_names))]
labels = labels.reset_index()
labels.head()

,index,id,breed
0,9,0042188c895a2f14ef64a918ed9c7b64,scottish_deerhound
1,12,00693b8bc2470375cc744a6391d397ec,maltese_dog
2,79,01e787576c003930f96c966f9c3e1d44,scottish_deerhound
3,80,01ee3c7ff9bcaba9874183135877670e,entlebucher
4,88,021b5a49189665c0442c19b5b33e8cf1,entlebucher


In [5]:
X_data = np.zeros((len(labels), 224, 224, 3), dtype='float32')
Y_data = label_binarize(labels['breed'], classes=breed_names)

for i in tqdm(range(len(labels))):
    img = image.load_img('/Users/nikhil/Downloads/archive/train/%s.jpg' % labels['id'][i], target_size=(224, 224) )
    img = image.img_to_array(img)
    x = np.expand_dims(img.copy(), axis=0)
    X_data[i] = x/255.0
    
print('/nTrain image shape', X_data.shape, 'size: {:,}'.format(X_data.size))
print('One hot encoded shape', Y_data.shape, 'size: {:,}'.format(X_data.size))

100%|████████████████████████████████████████| 588/588 [00:03<00:00, 152.31it/s]

/nTrain image shape (588, 224, 224, 3) size: 88,510,464
One hot encoded shape (588, 5) size: 88,510,464


In [6]:
model = Sequential()

# model.add(Conv2D(filters=128, kernel_size=(7,7), activation='relu', input_shape=(224,224,3)))
# model.add(MaxPool2D(pool_size=(2,2)))

model.add(Conv2D(filters=64, kernel_size=(5,5), activation='relu', input_shape=(224,224,3)))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Conv2D(filters=32, kernel_size=(3,3), activation='relu', kernel_regularizer='l2'))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Conv2D(filters=16, kernel_size=(7,7), activation='relu', kernel_regularizer='l2'))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Conv2D(filters=8, kernel_size=(5,5), activation='relu', kernel_regularizer='l2'))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(128, activation='relu', kernel_regularizer='l2'))
model.add(Dense(64, activation='relu', kernel_regularizer='l2'))
model.add(Dense(len(breed_names), activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer=Adam(0.00001), metrics=['accuracy'])

model.summary()

2022-10-03 15:26:23.680381: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 220, 220, 64)      4864      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 110, 110, 64)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 108, 108, 32)      18464     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 54, 54, 32)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 48, 48, 16)        25104     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 24, 24, 16)       0

In [7]:
X_train_val, X_test, Y_train_val, Y_test = train_test_split(X_data, Y_data, test_size=0.1)

X_train, X_val, Y_train, Y_val = train_test_split(X_train_val, Y_train_val, test_size=0.2)

In [ ]:
#training data
epochs = 100
batch_size = 128

fitted_model = model.fit(X_train, Y_train, batch_size=batch_size, epochs=epochs,
                        validation_data= (X_val, Y_val))

Epoch 1/100
4/4 [==============================] - 60s 12s/step - loss: 5.4163 - accuracy: 0.2033 - val_loss: 5.4122 - val_accuracy: 0.1604
Epoch 2/100
4/4 [==============================] - 48s 11s/step - loss: 5.4110 - accuracy: 0.2246 - val_loss: 5.4073 - val_accuracy: 0.1887
Epoch 3/100
4/4 [==============================] - 68s 18s/step - loss: 5.4059 - accuracy: 0.2270 - val_loss: 5.4024 - val_accuracy: 0.1981
Epoch 4/100
4/4 [==============================] - 59s 13s/step - loss: 5.4009 - accuracy: 0.2364 - val_loss: 5.3977 - val_accuracy: 0.1981
Epoch 5/100
4/4 [==============================] - 62s 14s/step - loss: 5.3959 - accuracy: 0.2388 - val_loss: 5.3929 - val_accuracy: 0.2075
Epoch 6/100
4/4 [==============================] - 60s 14s/step - loss: 5.3909 - accuracy: 0.2577 - val_loss: 5.3882 - val_accuracy: 0.2547
Epoch 7/100
4/4 [==============================] - 62s 14s/step - loss: 5.3859 - accuracy: 0.2648 - val_loss: 5.3834 - val_accuracy: 0.2547
Epoch 8/100
4/4 [===

4/4 [==============================] - 49s 12s/step - loss: 5.1393 - accuracy: 0.2908 - val_loss: 5.1462 - val_accuracy: 0.2830
Epoch 60/100
4/4 [==============================] - 49s 11s/step - loss: 5.1347 - accuracy: 0.2931 - val_loss: 5.1415 - val_accuracy: 0.2925
Epoch 61/100
4/4 [==============================] - 50s 12s/step - loss: 5.1301 - accuracy: 0.2955 - val_loss: 5.1372 - val_accuracy: 0.2830
Epoch 62/100
4/4 [==============================] - 49s 12s/step - loss: 5.1254 - accuracy: 0.2931 - val_loss: 5.1329 - val_accuracy: 0.2736
Epoch 63/100
4/4 [==============================] - 49s 12s/step - loss: 5.1209 - accuracy: 0.2931 - val_loss: 5.1286 - val_accuracy: 0.2736
Epoch 64/100
4/4 [==============================] - 51s 12s/step - loss: 5.1162 - accuracy: 0.2884 - val_loss: 5.1240 - val_accuracy: 0.2736
Epoch 65/100
4/4 [==============================] - 50s 12s/step - loss: 5.1115 - accuracy: 0.2931 - val_loss: 5.1190 - val_accuracy: 0.2830
Epoch 66/100
4/4 [========

In [ ]:
plt.figure(figsize=(12,5))
plt.plot(fitted_model.history['accuracy'], color='r')
plt.plot(fitted_model.history['val_accuracy'], color='b')
plt.legend(['train', 'val'])
plt.show()

In [ ]:
Y_pred = model.predict(X_test)
score = model.evaluate(X_test, Y_test)
print(round((score[1]*100), 2), '%')

In [ ]:
plt.imshow(X_test[1,:,:,:])
plt.show()
print('Originally: ' , labels['breed'][np.argmax(Y_test[1])])
print('Predicted: ' , labels['breed'][np.argmax(Y_pred[1])])